In [1]:
# Imports
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# Load the English STSB dataset
stsb_dataset = load_dataset('stsb_multi_mt', 'en')
stsb_train = pd.DataFrame(stsb_dataset['train'])
stsb_test = pd.DataFrame(stsb_dataset['test'])

# Check loaded data
print(stsb_train.shape, stsb_test.shape)  #gives the size of the data
stsb_test.head()

Reusing dataset stsb_multi_mt (C:\Users\hp\.cache\huggingface\datasets\stsb_multi_mt\en\1.0.0\a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


  0%|          | 0/3 [00:00<?, ?it/s]

(5749, 3) (1379, 3)


,sentence1,sentence2,similarity_score
0,A girl is styling her hair.,A girl is brushing her hair.,2.5
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2
4,A man is playing a harp.,A man is playing a keyboard.,1.5


In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load("en_core_web_sm")

def text_processing(sentence):
    """
    Lemmatize, lowercase, remove numbers and stop words
    
    Args:
      sentence: The sentence we want to process.
    
    Returns:
      A list of processed words
    """
    sentence = [token.lemma_.lower()          
                for token in nlp(sentence) 
                    if token.is_alpha and not token.is_stop]
    
    return sentence

#def cos_sim(sentence1_emb, sentence2_emb):
    """
    Cosine similarity between two columns of sentence embeddings
    
    Args:
      sentence1_emb: sentence1 embedding column
      sentence2_emb: sentence2 embedding column
    
    Returns:
      The row-wise cosine similarity between the two columns.
      For instance is sentence1_emb=[a,b,c] and sentence2_emb=[x,y,z]
      Then the result is [cosine_similarity(a,x), cosine_similarity(b,y), cosine_similarity(c,z)]
    """
 #   cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
  #  return np.diag(cos_sim)


In [3]:
import textdistance

def jaccard_sim(row):
    # Text Processing
    sentence1 = text_processing(row['sentence1'])
    sentence2 = text_processing(row['sentence2'])
    
    # Jaccard similarity
    return textdistance.jaccard.normalized_similarity(sentence1, sentence2)


# Jaccard Similarity
stsb_test['Jaccard_score'] = stsb_test.progress_apply(jaccard_sim, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 1379/1379 [00:35<00:00, 38.79it/s]


In [4]:

score_cols = [col for col in stsb_test.columns if '_score' in col]
print(score_cols)

#Spearman Rank Correlation
#spearman_rank_corr = stsb_test[score_cols].corr(method='spearman').iloc[1:, 0:1]*100
#spearman_rank_corr.head(10)

['similarity_score', 'Jaccard_score']


In [8]:
stsb_test.head()

,sentence1,sentence2,similarity_score,Jaccard_score
0,A girl is styling her hair.,A girl is brushing her hair.,2.5,0.500000
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6,0.666667
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0,1.000000
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2,0.500000
4,A man is playing a harp.,A man is playing a keyboard.,1.5,0.500000
